In [1]:
# ! pip install opencv-python matplotlib numpy

In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from util import get_parking_spots_bboxes, empty_or_not

def calc_diff(im1, im2):
    return np.abs(np.mean(im1) - np.mean(im2))

mask = './mask_1920_1080.png'
video_path = '../data/parking_1920_1080_loop.mp4'
mask = cv2.imread(mask, 0)

In [ ]:
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Erreur d'ouverture du fichier vidéo")
    exit()
connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)


In [5]:
spots = get_parking_spots_bboxes(connected_components)
spots_status = [None for j in spots]
diffs = [None for j in spots]
previous_frame = None

In [ ]:
frame_nmr = 0
ret = True
step = 30
while ret:
    ret, frame = cap.read()
    if not ret or frame is None:
        print('impossible de lire la vidéo')
        break
    if frame_nmr % step == 0 and previous_frame is not None:
        for spot_indx, spot in enumerate(spots):
            x1, y1, w, h = spot

            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]

            diffs[spot_indx] = calc_diff(spot_crop, previous_frame[y1:y1 + h, x1:x1 + w, :])

        print([diffs[j] for j in np.argsort(diffs)][::-1])

    if frame_nmr % step == 0:
        if previous_frame is None:
            arr_ = range(len(spots))
        else:
            arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.amax(diffs) > 0.4]
        for spot_indx in arr_:
            spot = spots[spot_indx]
            x1, y1, w, h = spot

            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]

            spot_status = empty_or_not(spot_crop)

            spots_status[spot_indx] = spot_status

    if frame_nmr % step == 0:
        previous_frame = frame.copy()

    for spot_indx, spot in enumerate(spots):
        spot_status = spots_status[spot_indx]
        x1, y1, w, h = spots[spot_indx]

        if spot_status:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
        else:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)

    cv2.rectangle(frame, (80, 20), (550, 80), (0, 0, 0), -1)
    cv2.putText(frame, 'Available spots: {} / {}'.format(str(sum(spots_status)), str(len(spots_status))), (100, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    frame_nmr += 1

cap.release()
cv2.destroyAllWindows()


[np.float64(7.591036414565821), np.float64(7.32528180354268), np.float64(7.1615858737298055), np.float64(7.014159586873248), np.float64(6.439095928226351), np.float64(6.401851851851859), np.float64(6.110378634212296), np.float64(6.088122605363992), np.float64(6.0571895424836555), np.float64(5.906423331544119), np.float64(5.861111111111114), np.float64(5.743794769282033), np.float64(5.742295518907213), np.float64(5.624070317782298), np.float64(5.597826086956523), np.float64(5.5755772005772), np.float64(5.574183006535947), np.float64(5.572663139329805), np.float64(5.556302521008391), np.float64(5.540896218557393), np.float64(5.507366035292208), np.float64(5.419623521572547), np.float64(5.369315342328846), np.float64(5.301759834368532), np.float64(5.296618357487915), np.float64(5.263235294117649), np.float64(5.246798029556643), np.float64(5.236392914653791), np.float64(5.215059137098109), np.float64(5.198951994590942), np.float64(5.186162870945481), np.float64(5.165307971014485), np.float